if needed, install the following library

In [ ]:
#!pip install py-algorand-sdk

In [1]:
import os, json, algosdk
from algosdk import account, mnemonic

## Create an Algorand account

Create a function to generate each account

In [2]:

def generate_account_dict():
    private_key = account.generate_account()[0]    # need [0], because generate_account() returns a list
    acc = {}
    acc['public'] = account.address_from_private_key(private_key)
    acc['private'] = private_key
    acc['mnemonic'] = mnemonic.from_private_key(private_key)
    return acc

Create an account for each user

In [3]:

Alfa = generate_account_dict()
Bravo   = generate_account_dict()
Charlie = generate_account_dict()

### Create the file structure

This is the first step to store your credentials

**WARNING**: 
Do NOT insert your credentials here, nor EVER print your private key or mnemonic in any jupyter notebook. Insert your credentials directly into the credentials file

In [4]:

cred = {'algod_test' : 'https://testnet-api.algonode.cloud',
        'algod_main' : 'https://mainnet-api.algonode.cloud',
        'index_test' : 'https://testnet-idx.algonode.cloud',
        'index_main' : 'https://mainnet-idx.algonode.cloud',
        'explore_main' : 'https://explorer.perawallet.app/',
        'explore_test' : 'https://testnet.explorer.perawallet.app/',
        'api_token'  : '',
        'pinata_jwt' : 'your pinata json web token here',
        'pinata_secret' : 'your pinata API secret here',
        'MyAlgo' : {'public' : '', 'private' : '', 'mnemonic' : 'your mnemonic here'}
        }
cred['Alfa'] = Alfa
cred['Bravo'] = Bravo
cred['Charlie'] = Charlie

Run the next cell to save the credentials to a file called credentials_temp. This will be created in the same directory as this notebook.

In [5]:
cred_json = json.dumps(cred,indent=4)

filename = 'credentials_temp'                   # op.path is needed to run on Win,Mac and Linux
with open(filename, 'w') as outfile:            # option 'w' ensures overwriting of existing file if it exists
    outfile.write(cred_json)

⚠️ **Before running the next cell, insert the mnemonic of your Algorand account into the credentials_temp file**

The mnemonic is needed to generate the private and public key for the MyAlgo account we will create in this next cell.

In [6]:
# read credentials file and saves them into the cred dictionary
with open(filename) as json_file:
    cred = json.load(json_file)

# calculate private and public key and save them in the cred dictionary under the correct key
cred['MyAlgo']['private'] = algosdk.mnemonic.to_private_key(cred['MyAlgo']['mnemonic'])
cred['MyAlgo']['public'] = account.address_from_private_key(cred['MyAlgo']['private'])

# write again the cred dictionary into the credentials file to store them
cred_json = json.dumps(cred,indent=4)                                    
with open(filename, 'w') as outfile:            # option 'w' ensures overwriting of existing file
    outfile.write(cred_json)

Before running the next cells, go to the appropriate folder and change the name of the file from `credentials_temp` to `credentials`.

Now we read the credentials from the file we just finalized.

In [7]:
import os, json

filename = 'credentials'
with open(filename) as json_file:
    cred = json.load(json_file)

Let's check if everything went well

In [8]:
# print Alfa public credentials
print(cred['Alfa']['public'])

B33ZV6UPJZXHBWWM65Y3ZWP4YGB7GLM2PR6WPMH3D6G674DWGDHYJXZIFE
https://explorer.perawallet.app/


In [9]:
# print Bravo public credentials
print(cred['Bravo']['public'])

ZEJV4X7XBW4FB55ZETC5XBJ7OV5IF3ZAX25IQRA7T5J5XM3II76JGAGJ7E


Let's first load a package of functions we will need to work with the Algorand Blockchain. This file was downloaded together with this notebook from https://github.com/nicarob/Lupoz-and-Gift-Crypto-Project. Don't move any files after the download of the whole folder, or it will not work.

In [10]:
# Loading the sharedCode functions
import sys, os
codepath = 'sharedCode'
sys.path.append(codepath)   # this adds the sharedCode folder to the list of folders where Python looks for modules
from algo_util import *     # this imports all functions from the sharedCode folder


Now let's load the credentials for each account

In [11]:
cred = load_credentials()
MyAlgo  = cred['MyAlgo']
Alfa    = cred['Alfa']
Bravo   = cred['Bravo']
Charlie = cred['Charlie']

Now let's import the relevant modules we will need in the next steps. We should already have 'algosdk' installed, if you don't have it yet, you can run the installation from the top of this notebook.

In [12]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.transaction import PaymentTxn
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
import algosdk.error
import json

**Initialize the algod client (Mainnet)**

In [13]:
# define some parameters needed to initialize the AlgodClient
algod_token   = ''                        
algod_address = cred['algod_main']        
api_token = cred['api_token']             

# Initialize the algod client
algod_client = algod.AlgodClient(algod_token=algod_token, algod_address=algod_address, headers=api_token)

# check last round to see if everything went well
algod_client.status()['last-round']

39977057

You can check if the transactions we will perform later are going correctly at the following links:

In [14]:
# Create a link to directly access your MyAlgo account
print(cred['explore_main']+'address/'+MyAlgo['public'])

https://explorer.perawallet.app/address/4MDVXHXTQIR7SXGGF62DPXUXSOLUEAFY75F6MHX35GV2X5NKIE2H53VISY


In [15]:
# Create a link to directly access your Alfa account
print(cred['explore_main']+'address/'+Alfa['public'])

https://explorer.perawallet.app/address/B33ZV6UPJZXHBWWM65Y3ZWP4YGB7GLM2PR6WPMH3D6G674DWGDHYJXZIFE


In [16]:
# Create a link to directly access your Bravo account
print(cred['explore_main']+'address/'+Bravo['public'])

https://explorer.perawallet.app/address/ZEJV4X7XBW4FB55ZETC5XBJ7OV5IF3ZAX25IQRA7T5J5XM3II76JGAGJ7E


In [17]:
# Create a link to directly access your Charlie account
print(cred['explore_main']+'address/'+Charlie['public'])

https://explorer.perawallet.app/address/SDDSAGSVMRK7FM5BFGUA7EHILEK3UQTDUBRJNZBZ3CJ6MDOKT6QB2BEVEY


## Create an ASA = Algorand Standard Asset - Lupoz

* Each Lupoz has 6 decimals
    * Hence, our token can be divided into units as small as $\frac{1}{10^6} = \frac{1}{1000000} = 0.000001$
* To create 1000 tokens, we must create $1000 \cdot 10^6$ *microLupoz*

In [67]:
# Let's define the variables needed to create the new asset

sp = algod_client.suggested_params()
token_supply = 1000                              # Token supply (big units)
token_decimals =  6                              # Digits after the comma
token_total = token_supply * 10**token_decimals  # Specify number of SMALLER unit ("microLupoz")
print(token_total)

token_name  = "Lupoz"                         
token_url   = "https://github.com/nicarob/Lupoz-and-Gift-Crypto-Project"                   
token_unit  = "LUP"                              

1000000000


During creation, we need to assign the following roles, which we will use later ...

* **Manager:** can change the the follwoing roles and can destroy the asset
* **Reserve:** where not yet distributed assets reside (informational)
* **Freeze:** can freeze assets (e.g. to wait for KYC)
* **Clawback:** can undo transactions

See https://developer.algorand.org/docs/features/asa/ 

In [68]:
txn = AssetConfigTxn(
    sender=MyAlgo['public'],                   # Creator of the ASA
    sp=sp,                      
    total=token_total,                         # Total supply in SMALL unit
    decimals=token_decimals,                   # Decimals
    default_frozen=False,                      # Tokens are not frozen by default
    unit_name=token_unit,                      # Abbreviation     
    asset_name=token_name,                     # Name
    url=token_url,                             # URL
    manager=MyAlgo['public'],             
    reserve=MyAlgo['public'],                
    freeze=MyAlgo['public'],                 
    clawback=MyAlgo['public']               
)
print(txn)

{'sender': '4MDVXHXTQIR7SXGGF62DPXUXSOLUEAFY75F6MHX35GV2X5NKIE2H53VISY', 'fee': 1000, 'first_valid_round': 39978275, 'last_valid_round': 39979275, 'note': None, 'genesis_id': 'mainnet-v1.0', 'genesis_hash': 'wGHE2Pwdvd7S12BL5FaOP20EGYesN73ktiC1qzkkit8=', 'group': None, 'lease': None, 'type': 'acfg', 'rekey_to': None, 'index': 0, 'total': 1000000000, 'default_frozen': False, 'unit_name': 'LUP', 'asset_name': 'Lupoz', 'manager': '4MDVXHXTQIR7SXGGF62DPXUXSOLUEAFY75F6MHX35GV2X5NKIE2H53VISY', 'reserve': '4MDVXHXTQIR7SXGGF62DPXUXSOLUEAFY75F6MHX35GV2X5NKIE2H53VISY', 'freeze': '4MDVXHXTQIR7SXGGF62DPXUXSOLUEAFY75F6MHX35GV2X5NKIE2H53VISY', 'clawback': '4MDVXHXTQIR7SXGGF62DPXUXSOLUEAFY75F6MHX35GV2X5NKIE2H53VISY', 'url': 'https://github.com/nicarob/Lupoz-and-Gift-Crypto-Project', 'metadata_hash': None, 'decimals': 6}


Sign and send the transaction

In [69]:
stxn = txn.sign(MyAlgo['private'])             # Sign
txid = algod_client.send_transaction(stxn)     # Send
print(txid)


7GNVD345BCXOREPKKP3KT4L2NDYVU6GJKLAGGCMCH3V3OJ7FJPXA


Wait for the transaction to be confirmed

In [70]:
txinfo = wait_for_confirmation(algod_client,txid)

Current round is  39978276.
Waiting for round 39978276 to finish.
Waiting for round 39978277 to finish.
Transaction 7GNVD345BCXOREPKKP3KT4L2NDYVU6GJKLAGGCMCH3V3OJ7FJPXA confirmed in round 39978278.


Get the asset index and information to check if everything is fine

In [71]:
print(txinfo)

{'asset-index': 2039533297, 'confirmed-round': 39978278, 'pool-error': '', 'txn': {'sig': 'psUwwLvwkLmsonuSDbQkGJULAY6w7uyfe009aEkkLa1OdNNh7VYK3Iq5yIOL8SInwRKvu1lCXY0GPb9eG8dpAw==', 'txn': {'apar': {'an': 'Lupoz', 'au': 'https://github.com/nicarob/Lupoz-and-Gift-Crypto-Project', 'c': '4MDVXHXTQIR7SXGGF62DPXUXSOLUEAFY75F6MHX35GV2X5NKIE2H53VISY', 'dc': 6, 'f': '4MDVXHXTQIR7SXGGF62DPXUXSOLUEAFY75F6MHX35GV2X5NKIE2H53VISY', 'm': '4MDVXHXTQIR7SXGGF62DPXUXSOLUEAFY75F6MHX35GV2X5NKIE2H53VISY', 'r': '4MDVXHXTQIR7SXGGF62DPXUXSOLUEAFY75F6MHX35GV2X5NKIE2H53VISY', 't': 1000000000, 'un': 'LUP'}, 'fee': 1000, 'fv': 39978275, 'gen': 'mainnet-v1.0', 'gh': 'wGHE2Pwdvd7S12BL5FaOP20EGYesN73ktiC1qzkkit8=', 'lv': 39979275, 'snd': '4MDVXHXTQIR7SXGGF62DPXUXSOLUEAFY75F6MHX35GV2X5NKIE2H53VISY', 'type': 'acfg'}}}


Get the asset ID and open in Algoexplorer

In [72]:
lupoz_index = txinfo['asset-index']

In [73]:
print(lupoz_index)
print(cred['explore_main']+'asset/{}'.format(lupoz_index))

2039533297
https://explorer.perawallet.app/asset/2039533297


## The Smart signature

We will use pyTeal to implement the logic of the smart signature.

Uncomment and install if needed:

In [74]:
#!pip3 install pyteal

In [75]:
from pyteal import * # type: ignore
import datetime
import math

### Let's define the logic of the smart signature:
- the logic concerning the present itself
- the logic concerning the opt-in of the signature to Lupoz
- the logic concerning security
- the logic concerning transaction fees

Let's first create a function to generate the correct logic for the present:

In [98]:
def present(receiver, day, month, year, amount):
    '''
    Create a smart signature to allow Bravo to redeem a present

    Args:
    receiver (str): The public address of the receiver (Bravo)
    year (int): The year in which you want to do the present
    amount (float): The amount of Algo to be sent

    Returns:
    A PyTeal expression that checks the conditions for the present to be redeemed.
    '''

    # Define the birthday date in Unix timestamp
    present =  math.floor(datetime.datetime(year, month, day).timestamp())
    
    receiver_pyteal = Addr(receiver)

    # Define the amount of Algo to be sent (converted to microAlgos)
    microlupoz_int = math.floor(amount*1e6)
    microlupoz = Int(microlupoz_int)

    # This next part roughly calculates the block number corresponding to the specified date. 
    # You cannot use the date directly in a smart signature, so we had to resort to this workaround. 
    
    # Define the timestamp of the genesis block
    ALGOD_GENESIS_TIMESTAMP = 1560902400
    # Calculate the number of seconds since the genesis block
    seconds_since_genesis = present - ALGOD_GENESIS_TIMESTAMP
    # This is the average time between blocks in seconds
    ALGOD_BLOCK_INTERVAL = 4.5
    # Calculate the corresponding Algorand round
    round_number = int(seconds_since_genesis / ALGOD_BLOCK_INTERVAL)

    return And(
        Txn.type_enum() == TxnType.AssetTransfer,                 # <-- Verify this is a payment transaction
        Txn.asset_receiver() == receiver_pyteal,            # <-- Verify that the receiver is Bravo 
        Txn.xfer_asset() == Int(lupoz_index),               # <-- Check that the coin has index equal to the index of the coin we created
        Int(round_number) <= Int(present),                  # <-- Bravo can redeem the present only from the specified date onwards 
        Txn.amount() <= microlupoz                          # <-- The amount must be exactly the specified amount
        )

Now let's use that function to implement our specific case

In [99]:
receiver = cred['Bravo']['public']
day = 25
month = 5
year = 2024
amount = 10

# define the logic of the present itself
present_logic = present(receiver, day, month, year, amount)


Now the other logic needed

In [100]:
security_logic = And(
    Txn.close_remainder_to() == Global.zero_address(),  # <-- Closer attack!
    Txn.rekey_to() == Global.zero_address(),            # <-- Rekeying attack!
    )

fee_logic = And(
    Txn.fee() <= Int(1000)                              # <-- Fee must not be over 1000 microAlgos
    )


opt_in_logic = And(
    Txn.asset_receiver() == Txn.sender(),                  # <-- The receiver of the coin must be the sender
    Txn.xfer_asset() == Int(lupoz_index))                  # <-- The coin must have the index of the coin we created


Let's put them together to get the final logic of the smart signature we are creating

In [101]:
full_logic = And(
    security_logic,
    fee_logic,
    Or(
    opt_in_logic, 
    present_logic
    )
)

Compile the PyTeal program to TEAL

In [102]:

#The if __name__ == "__main__": construct is a common idiom in Python that allows you to run code only when the script is executed directly, and not when it is imported as a module in another script
if __name__ == "__main__":

    
    compiled_teal = compileTeal(full_logic, mode=Mode.Signature, version=2)

Submit the signature to the Algorand blockchain

In [103]:

present_sig = algod_client.compile(compiled_teal)

Let's import some other modules that will be needed in this next section

In [104]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk import transaction
from algosdk.transaction import PaymentTxn
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.transaction import LogicSig, LogicSigTransaction

import algosdk.error
import json
import base64
import hashlib

## The transactions to set every account up correctly
We need to fund each account involved with some Algos and Lupoz, in order to smoothly complete the transactions for the gift.

Each account needs to pay 1000 microAlgos of fee to opt-in to Lupoz, and to send Lupoz to the smart signature. Note that the smart signature itself must also opt-in, so we need to fund it accordingly. There are minimum requirements concerning Algos for each account; our transactions take that into account.

In our case, Alfa and Charlie are together making the present, providing 5 Lupoz each. Bravo will be able to redeem the 10 Lupoz present after the specified date.

**1) MyAlgo funds Alfa and Charlie with 5000 microAlgos**

In [105]:
# MyAlgo funds Alfa

# Step 1: prepare and create TX
sp = algod_client.suggested_params()             # Suggested params
amt = int(500000)                                     # <-- Send 5000 microAlgos

txn = transaction.PaymentTxn(
    sender=MyAlgo['public'],
    sp=sp,
    receiver=Alfa['public'],
    amt=amt
    )                

# Step 2+3: sign + send
signed_txn = txn.sign(MyAlgo['private'])
txid = algod_client.send_transaction(signed_txn)
    
# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client,txid)

Current round is  39978355.
Waiting for round 39978355 to finish.
Waiting for round 39978356 to finish.
Transaction OYH53RXIHK4AAVJXBMWH2I3JLNIP574EESRZIT2QYKV6AFKVVYGQ confirmed in round 39978357.


In [106]:
# MyAlgo funds Charlie

# Step 1: prepare and create TX
sp = algod_client.suggested_params()             # Suggested params
amt = int(500000)                                     # <-- Send 5000 microAlgos

txn = transaction.PaymentTxn(
    sender=MyAlgo['public'],
    sp=sp,
    receiver=Charlie['public'],
    amt=amt
    )                

# Step 2+3: sign + send
signed_txn = txn.sign(MyAlgo['private'])
txid = algod_client.send_transaction(signed_txn)
    
# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client,txid)

AlgodHTTPError: TransactionPool.Remember: transaction EZYDCUY7RAKNJEPI3GDYWMK3D26KFGKY73Y7ICOXSUZRCK7D6KSA: account 4MDVXHXTQIR7SXGGF62DPXUXSOLUEAFY75F6MHX35GV2X5NKIE2H53VISY balance 229000 below min 400000 (3 assets)

**2) Alfa and Charlie opt-in to Lupoz**

In [107]:
# Opt-in for Alfa

# Step 1: prepare and create TX
sp = algod_client.suggested_params()             # Suggested params
optin_amt = int(0)                                     # <-- Send 0 token

txn = AssetTransferTxn(
    sender=Alfa['public'],
    sp=sp,
    receiver=Alfa['public'],
    amt=optin_amt,
    index=lupoz_index
    )                

# Step 2+3: sign + send
signed_txn = txn.sign(Alfa['private'])
txid = algod_client.send_transaction(signed_txn)
    
# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client,txid)

Current round is  39978395.
Waiting for round 39978395 to finish.
Transaction WAOBGJN44WD4RS3FMQHB56VIDLAAQLBGBPTZCTUDZOMHOBTSCFMQ confirmed in round 39978396.


In [108]:
# Opt-in for Charlie

# Step 1: prepare and create TX
sp = algod_client.suggested_params()             # Suggested params
optin_amt = int(0)                                     # <-- Send 0 token

txn = AssetTransferTxn(
    sender=Charlie['public'],
    sp=sp,
    receiver=Charlie['public'],
    amt=optin_amt,
    index=lupoz_index
    )                

# Step 2+3: sign + send
signed_txn = txn.sign(Charlie['private'])
txid = algod_client.send_transaction(signed_txn)
    
# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client,txid)

Current round is  39978396.
Waiting for round 39978396 to finish.
Waiting for round 39978397 to finish.
Transaction EXQAF7V7QMOOHOPIS443EAT3HRNXVWJPO2M6HWFUUAEB5OXL3KPA confirmed in round 39978398.


**3) MyAlgo funds Alfa and Charlie with appropiate amount of Lupoz**

In [109]:
# MyAlgo funds Alfa with 5 lupoz tokens

# Step 1: prepare and create TX
sp = algod_client.suggested_params()             # Suggested params
amt = int(5*1e6)                                     # <-- Send 5 token

txn = AssetTransferTxn(
    sender=MyAlgo['public'],
    sp=sp,
    receiver=Alfa['public'],
    amt=amt,
    index=lupoz_index
    )                

# Step 2+3: sign + send
signed_txn = txn.sign(MyAlgo['private'])
txid = algod_client.send_transaction(signed_txn)
    
# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client,txid)

Current round is  39978399.
Waiting for round 39978399 to finish.
Waiting for round 39978400 to finish.
Transaction 5CWXEJUGBFJ7IVKB4YLVATCC7B6ZUAZPN7IWTDFXPND4HGYU3UXQ confirmed in round 39978401.


In [110]:
# MyAlgo funds Charlie with 5 lupoz tokens

# Step 1: prepare and create TX
sp = algod_client.suggested_params()             # Suggested params
amt = int(5*1e6)                                     # <-- Send 5 token

txn = AssetTransferTxn(
    sender=MyAlgo['public'],
    sp=sp,
    receiver=Charlie['public'],
    amt=amt,
    index=lupoz_index
    )                

# Step 2+3: sign + send
signed_txn = txn.sign(MyAlgo['private'])
txid = algod_client.send_transaction(signed_txn)
    
# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client,txid)

Current round is  39978402.
Waiting for round 39978402 to finish.
Waiting for round 39978403 to finish.
Transaction FKJWE2DOLT4DW7DX2ZH3INVWQGRA5TQTIJYG76IN7OYCECYRRNPA confirmed in round 39978404.


**4) MyAlgo funds Bravo to pay for the opt-in & for the transaction fees when he redeems the present**

In [111]:
# MyAlgo funds Bravo
# Step 1: prepare and create TX
sp = algod_client.suggested_params()             # Suggested params
amt = int(250000)                                     # <-- Send 2500 microAlgos

txn = transaction.PaymentTxn(
    sender=MyAlgo['public'],
    sp=sp,
    receiver=Bravo['public'],
    amt=amt
    )                

# Step 2+3: sign + send
signed_txn = txn.sign(MyAlgo['private'])
txid = algod_client.send_transaction(signed_txn)
    
# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client,txid)

Current round is  39978409.
Waiting for round 39978409 to finish.
Waiting for round 39978410 to finish.
Transaction QW23XV5HX6TKYEHDCOSVPEG5EYWHI2GZABVPFOEBZUMUGRZJDQFA confirmed in round 39978411.


**5) Bravo opts-in to Lupoz**

In [112]:
# Opt-in for Bravo

# Step 1: prepare and create TX
sp = algod_client.suggested_params()             # Suggested params
optin_amt = int(0)                                     # <-- Send 0 token

txn = AssetTransferTxn(
    sender=Bravo['public'],
    sp=sp,
    receiver=Bravo['public'],
    amt=optin_amt,
    index=lupoz_index
    )                

# Step 2+3: sign + send
signed_txn = txn.sign(Bravo['private'])
txid = algod_client.send_transaction(signed_txn)
    
# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client,txid)

Current round is  39978414.
Waiting for round 39978414 to finish.
Transaction N7FV2PVRIAQTBAILNXUTV4QLIR4B7URAQUDPIL3RRWIZFQWNIZGA confirmed in round 39978415.


## The actual smart signature transaction

As mentioned before, the signature must also pay fees to opt-in to Lupoz.

**1) Alfa funds the smart signature**

In [113]:
#Alfa funds the smart signature 

# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int(250000)
txn = transaction.PaymentTxn(sender=Alfa['public'], sp=sp, receiver=present_sig['hash'], amt=amt)

# Step 2+3: sign and send
stxn = txn.sign(Alfa['private'])
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  39978415.
Waiting for round 39978415 to finish.
Waiting for round 39978416 to finish.
Transaction DQILLLU3NOHVNNI2HNXQLLH5UHWRIUKPHW3TZJVIDXGHY6AQYCPQ confirmed in round 39978417.


**2) The signature opts-in**

In [114]:

# Step 5.1: Prepare
sp = algod_client.suggested_params()
txn = AssetTransferTxn(present_sig['hash'], sp, present_sig['hash'], 0, lupoz_index)

# Step 5.2: Sign
encodedProg = present_sig['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 5.3 Send
txid = algod_client.send_transaction(stxn)

# Step 5.4 Wait for ...
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  39978417.
Waiting for round 39978417 to finish.
Waiting for round 39978418 to finish.
Transaction NQIH7DSUU4UIR5B5CQVRCZB7UGJ7KVC544XXR77WL5JNZXTPSUCQ confirmed in round 39978419.


**3) Alfa and Charlie fund the signature for the present**

In [115]:
# Alfa funds the signature

# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int(5*1e6)
txn = AssetTransferTxn(sender=Alfa['public'], sp=sp, receiver=present_sig['hash'], amt=amt, index=lupoz_index)

# Step 2+3: sign and send
stxn = txn.sign(Alfa['private'])
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  39978419.
Waiting for round 39978419 to finish.
Waiting for round 39978420 to finish.
Transaction PAOA562B6USDKRGZFHMAUSKS2BOVE4VQDMWWU7Q2L3TNVUCOXTTQ confirmed in round 39978421.


In [116]:
# Charlie funds the signature

# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int(5*1e6)
txn = AssetTransferTxn(sender=Charlie['public'], sp=sp, receiver=present_sig['hash'], amt=amt, index=lupoz_index)

# Step 2+3: sign and send
stxn = txn.sign(Charlie['private'])
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  39978421.
Waiting for round 39978421 to finish.
Waiting for round 39978422 to finish.
Transaction AZJ5XK4DH4UTORUO4RYBO4LXBJ244XFHD73YRQ2VFPN7E73POQAQ confirmed in round 39978423.


**4) Finally, Bravo redeems the present**

In [117]:
#Bravo redeems the present from the Smart Signature
# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int(10*1e6)
txn = AssetTransferTxn(sender=present_sig['hash'], sp=sp, receiver=Bravo['public'], amt=amt, index=lupoz_index)

# Step 2: sign TX
encodedProg = present_sig['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  39978423.
Waiting for round 39978423 to finish.
Waiting for round 39978424 to finish.
Transaction LA5I5UGAYM6ZGR3FR7Z3WVVCRFDI4IJEAALVWYI2BZ6QE3DHJBFQ confirmed in round 39978425.


Now we can check if everything went according to plans

In [ ]:
# MyAlgo 
print(cred['explore_main']+'address/'+MyAlgo['public'])

https://explorer.perawallet.app/address/4MDVXHXTQIR7SXGGF62DPXUXSOLUEAFY75F6MHX35GV2X5NKIE2H53VISY


In [ ]:
# Alfa 
print(cred['explore_main']+'address/'+Alfa['public'])

https://explorer.perawallet.app/address/B33ZV6UPJZXHBWWM65Y3ZWP4YGB7GLM2PR6WPMH3D6G674DWGDHYJXZIFE


In [ ]:
# Bravo 
print(cred['explore_main']+'address/'+Bravo['public'])

https://explorer.perawallet.app/address/ZEJV4X7XBW4FB55ZETC5XBJ7OV5IF3ZAX25IQRA7T5J5XM3II76JGAGJ7E


In [ ]:
# Charlie
print(cred['explore_main']+'address/'+Charlie['public'])

https://explorer.perawallet.app/address/SDDSAGSVMRK7FM5BFGUA7EHILEK3UQTDUBRJNZBZ3CJ6MDOKT6QB2BEVEY
